In [251]:
import pandas as pd
import json
import requests


documentation of pages
page:
1 Candidate
2 PAC
3 Political Party
4 Organizations
5 Independent Expenditures
6 Ballot Measures
7 Individual Contrbutors
8 Vendors

11 Name

20 Candidate/Income
21 Candidate/Expense
22 Candidate/IEFor
23 Candidate/IEAgainst
24 Candidate/All Transactions (appears to only contain income and expenses, not IE?)

30 PAC/Income
31 PAC/Expense
32 PAC/IEFor
33 PAC/IEAgainst
34 PAC/BMEFor
35 PAC/BMEAgainst
36 PAC/All Transactions

40 Political Party/Income
41 Political Party/Expense
42 Political Party/All Transactions

50 Organizations/IEFor
51 Organizations/IEAgainst
52 Organizations/BMEFor
53 Organizations/BME Against
54 Organizations/All Transactions

60 Independent Expenditures/IEFor
61 Independent Expenditures/IEAgaints
62 Independent Expenditures/All Transactions

70 Ballot Measures/Amount For
71 Ballot Measures/Amount Against
72 Ballot Measures/All Transactions

80 Individuals/All Transactions

90 Vendors/All Transactions



pages_dict = {

}




Looks like parameters are always the same, but the data changes by page. We can work with this, make a library of base templates for the data, keep headers the same probably, get rid of cookies, make parameters customizable





In [294]:
pages_dict = {
    "Candidate": 1,
    "PAC": 2,
    "Political Party": 3,
    "Organzations": 4,
    "Independent Expenditures": 5,
    "Ballot Measures": 6, 
    "Individual Contributors": 7,
    "Vendors": 8,
    
    "Name": 11,
    
    "Candidate/Income": 20,
    "Candidate/Expense": 21, 
    "Candidate/IEFor": 22,
    "Candidate/IEAgainst": 23,
    "Candidate/All Transactions":24,
    
    "PAC/Income": 30,
    "PAC/Expense": 31,
    "PAC/IEFor": 32,
    "PAC/IEAgainst": 33,
    "PAC/BMEFor": 34,
    "PAC/BMEAgainst": 35,
    "PAC/All Transactions": 36,
    
    "Political Party/Income": 40,
    "Political Party/Expense": 41,
    "Political Party/All Transactions": 42,

    "Organizations/IEFor": 50,
    "Organizations/IEAgainst": 51,
    "Organizations/BMEFor": 52,
    "Organizations/BME Against": 53,
    "Organizations/All Transactions": 54,
    
    "Independent Expenditures/IEFor": 60,
    "Independent Expenditures/IEAgainst": 61,
    "Independent Expenditures/All Transactions": 62,
    
    "Ballot Measures/Amount For": 70,
    "Ballot Measures/Amount Against": 71,
    "Ballot Measures/All Transactions": 72,
    
    "Individuals/All Transactions": 80,
    
    "Vendors/All Transactions": 90,
    
}


In [308]:
page = pages_dict["Vendors/All Transactions"]
page

90

In [309]:
page in [1, 2, 90]

True

## Functions

What does it not do right now?

It does not iterate, which makes scraping the individual dataset difficult, will have to build that in

It does not scrape the Name page. Since this is a non-tabular page it is currently outside of scope, but we may add this functionality later

It does not clean the data. Data cleaning will occur at a later stage

The high-level functions don't leave room to specify things like table page or table length. Maybe add those in as optional later, though likely not necessary, especially once we get iteration working

Currently the start and end dates are coarse, year by year, whereas our client likely wants quarterly scrapes. Look into getting this done later

We haven't looked into parameters like 'JurisdictionID', 'IssLessActive', and 'ShowOfficeHolder'

Haven't added error messages

In [326]:
def az_wrapper(page: str, start_year = 2023, end_year = 2023, *args, **kwargs: int) -> pd.DataFrame:
    """Take desired page, start and end years, and return the scraped data
    
    This function retrieves and compiles the data from a given table 
    from the arizona database, whether aggregate or detailed, within the given time period
    NOTE: Empty returns are to be expected for some inputs, as some tables are empty or 
    near empty even for long spans of time. 
    
    Args: page: the name of a basic or detailed page in the Arizona dataset, excluding the 
    Name page. 
    start_year: earliest year to include scraped data, inclusive
    end_year: last year to include scraped data, inclusive
    
    Returns: a pandas dataframe of the 
    """
    
    page = pages_dict[page]
    
    if page<10:
        return scrape_wrapper(page, start_year, end_year)
    
    else:
        
        agg_df = scrape_wrapper(detailed_wrapper_director(page), start_year, end_year)
        entities = entities = agg_df["EntityID"]
        
        return detailed_scrape_wrapper(entities, page, start_year, end_year)
    

def detailed_wrapper_director(page: int) -> int:
    """ Turn detailed page number into parent page number
    
    This function takes as input the number of the page input to 
    scrape_wrapper in the course of az_wrapper, and derives the 
    number of the parent page which must be scraped for 
    entities first
    
    Args: page: the two-digit page number belonging to a 
    detailed page as shown in pages_dict.
    
    Returns: an integer representing the parent page
    """
    
    if page in [20, 21, 22, 23, 24]:
        return 1
    
    elif page in [30, 31, 32, 33, 34, 35, 36]:
        return 2
    
    elif page in [40, 41, 42]:
        return 3
    
    elif page in [50, 51, 52, 53, 54]:
        return 4
    
    elif page in [60, 61, 62]:
        return 5
    
    elif page in [70, 71, 72]:
        return 6

    elif page in [80]:
        return 7
    
    elif page in [90]:
        return 8
    

    
def scrape_wrapper(page, start_year, end_year, *args: int) -> pd.DataFrame:
    """Create parameters and scrape an aggregate table
    
    This function is called by az_wrapper() to create the parameters and 
    call the basic scraper for a certain basic page. To scrape the detailed
    pages, use detailed_scrape_wrapper() instead.
    
    Args: page: the one-digit number representing one of the eight
    basic pages in the arizona dataset, such as Candidates, PAC, 
    Individual Contributions, etc. Refer to pages_dict
    start_year: earliest year to include scraped data, inclusive
    end_year: last year to include scraped data, inclusive
    
    Returns: a pandas dataframe containing the table data for
    the selected timeframe
    """
    params = parametrize(page, start_year, end_year)
    res = scrape(params, head, base1)
    results = res.json()
    df = pd.DataFrame(data=results["data"]).reset_index().drop(columns={"index"})
    return df

        
def detailed_scrape_wrapper(entities: pd.core.series.Series, page: int, start_year: int, end_year: int) -> pd.DataFrame:
    """Create parameters and scrape an aggregate table
    
    This function is called by az_wrapper() to create the parameters and 
    call the detailed scraper for a certain detailed page. To scrape the 
    basic pages, use scrape_wrapper() instead.
    
    Args: page: the two-digit number representing a sub-page of
    one of the eight basic pages, such as Candidates/Income, 
    PAC/All Transactions, etc. Refer to pages_dict
    start_year: earliest year to include scraped data, inclusive
    end_year: last year to include scraped data, inclusive
    
    Returns: a pandas dataframe containing the table data for
    the selected timeframe
    """
    
    d_params = []
    
    for entity in entities:
        d_params.append(detailed_parametrize(entity, page, start_year, end_year))
    
    dfs = []
    
    for d_param in d_params:
        res = detailed_scrape(d_param, start_year, end_year)
        results = res.json()
        dfs.append(pd.DataFrame(data=results["data"]))
    
    return pd.concat(dfs).reset_index().drop(columns={"index"})
                   
    
    
def scrape(params: dict, headers: dict, data: dict) -> requests.models.Response:
    """Scrape a table from the main arizona site
    
    This function takes in the header and base provided
    elsewhere, and parameters generated by parametrize(),
    to locate and scrape data from one of the eight
    aggregate tables on the Arizona database. 
    
    Args: params: created from parametrize(), containing
    the page, start and end years, table page, and table length.
    Note that 'page' encodes the page to be scraped, such as 
    Candidates, IndividualContributions, etc. Refer to the 
    attached Pages dictionary for details. 
    headers: necessary for calling the response, provided above
    data: necessary for calling the response, provided above
    """
    
    return requests.post(
        'https://seethemoney.az.gov/Reporting/GetNEWTableData/',
        params=params,
        #cookies=cookies,
        headers=head,
        data=base1)


def detailed_scrape(detailed_params: dict, headers: dict, data: dict) -> requests.models.Response:
    """Scrape a sub-table from the arizona database
    
    This function takes an entity number, which can be
    gathered from an aggregate table using scrape() or
    inputted manually, and gathers that entity's detailed
    information within the specified time frame from
    one of the sub-tables. 
    
    Args: detailed_params: created from detailed_parametrize(),
    containing the entity_id, page, start and end years, table page, and table length.
    Note that 'page' encodes the page to be scraped, such as 
    Candidates, IndividualContributions, etc. Refer to the 
    attached Pages dictionary for details. 
    headers: necessary for calling the response, provided above
    data: necessary for calling the response, provided above
    """
    
    return requests.post(
        'https://seethemoney.az.gov/Reporting/GetNEWDetailedTableData/',
        params=detailed_params,
        #cookies=cookies,
        headers=head,
        data=base1)



def parametrize(page = 1, start_year = 2023, end_year = 2025, table_page = 1, table_length = 500000, **kwargs: int) -> dict:
    """Input parameters for scrape and return as dict
    
    This function takes in parameters to scrape a given section of the arizona
    database, and turns them into a dictionary to be fed into scrape() as params
    
    Kwargs: page: encodes the page to be scraped, such as Candidates, Individual
    Contributions, etc. Refer to the attached Pages dictionary for details. 
    this should be backend, make a wrapper function that allows this to be
    done in a human-friendly way
    start_year: earliest year to include scraped data, inclusive
    end_year: last year to include scraped data, inclusive
    table_page: the numbered page to be accessed. Only necessary to iterate on this
    if accessing large quantities of Individual Contributions data,
    as all other data will be captured whole by the default table_length
    table_length: the length of the table to be scraped. The default setting should 
    scrape the entirety of the desired data unless looking at Individual Contributions
    
    Returns: a dictionary of the parameters, to be fed into scrape()
    """
    return {
        'Page': str(page), #refers to the overall page, like candidates or individual expenditures
        'startYear': str(start_year),
        'endYear': str(end_year),
        'JurisdictionId': '0|Page', #we keep this in here, but don't use it, not sure what it does?
        'TablePage': str(table_page),
        'TableLength': str(table_length),
    
        'ChartName': str(page),

        'IsLessActive': 'false', #have yet to experiment with these
        'ShowOfficeHolder': 'false', #have yet to experiment with these
    }



def detailed_parametrize(entity_id, page = 1, start_year = 2023, end_year = 2025, table_page = 1, table_length = 500000, *args: int, **kwargs: int) -> dict:
    """
    
    """
    return {
        'Page': str(page), #refers to the overall page, like candidates or individual expenditures
        'startYear': str(start_year),
        'endYear': str(end_year),
        'JurisdictionId': '0|Page', #we keep this in here, but don't use it, not sure what it does?
        'TablePage': str(table_page),
        'TableLength': str(table_length),
    
        'Name': '1~' + str(entity_id), #these two get used when scraping detailed data
        'entityId': str(entity_id),
    
        'ChartName': str(page),

        'IsLessActive': 'false', #have yet to experiment with these
        'ShowOfficeHolder': 'false', #have yet to experiment with these
    }

    
    

In [327]:
%%time
az_wrapper("PAC/IEFor", 2015, 2015)

CPU times: user 16.2 s, sys: 1.67 s, total: 17.9 s
Wall time: 2min 36s


,PublicTransactionId,TransactionId,TransactionDate,TransactionTypeId,TransactionTypeDispositionId,CommitteeId,CommitteeUniqueId,CommitteeName,Amount,EntityTypeId,...,TransactionState,TransactionZipCode,EntityDescription,Memo,TransactionType,JurisdictionId,JurisdictionName,ReportId,TransactionDateYear,BenefitedOpposed
0,7623234,6221035,/Date(1429599600000)/,33,2,201600068,31775,Total N-Tegrity Committee (Tnt),70.00,23,...,PA,19063,Business,None,Independent Expenditure - Pay Cash/Check,0,Arizona Secretary of State,None,2015,Benefited
1,7623236,6221049,/Date(1430290800000)/,33,2,201600068,31775,Total N-Tegrity Committee (Tnt),10.00,23,...,PA,19063,Business,None,Independent Expenditure - Pay Cash/Check,0,Arizona Secretary of State,None,2015,Benefited
2,7623237,6221044,/Date(1430463600000)/,33,2,201600068,31775,Total N-Tegrity Committee (Tnt),509.55,23,...,AZ,85713,Business,None,Independent Expenditure - Pay Cash/Check,0,Arizona Secretary of State,None,2015,Benefited
3,7623240,6221046,/Date(1432710000000)/,33,2,201600068,31775,Total N-Tegrity Committee (Tnt),700.00,23,...,AZ,85713,Business,None,Independent Expenditure - Pay Cash/Check,0,Arizona Secretary of State,None,2015,Benefited
4,2273498,6164110,/Date(1444719600000)/,33,2,1524,29540,United Phoenix Firefighters Peoria Chapter,250.00,21,...,AZ,85302,Individual,Campaign Contribution,Independent Expenditure - Pay Cash/Check,0,Arizona Secretary of State,None,2015,Benefited


In [253]:
par = parametrize(page=1,  start_year = 2021, end_year = 2022)
par

{'Page': '1',
 'startYear': '2021',
 'endYear': '2022',
 'JurisdictionId': '0|Page',
 'TablePage': '1',
 'TableLength': '500000',
 'ChartName': '1',
 'IsLessActive': 'false',
 'ShowOfficeHolder': 'false'}

In [258]:
len(ents)

511

In [280]:
res = scrape(par, head, base1)

In [281]:
results = res.json()
test_df2 = pd.DataFrame(data=results["data"])
test_df2#["EntityID"]#.dtype

""


In [273]:
%%time

pars = []
for ent in ents[0:10]:
    pars.append(detailed_parametrize(ent, page=24,  start_year = 2021, end_year = 2021))

dfs = []

for par in pars:
    res = detailed_scrape(par, head, base24)
    df = pd.DataFrame(data = res.json()["data"])
    dfs.append(df)

all_trans_cands = pd.concat(dfs)
all_trans_cands

#the detailed list doesn't have entity numbers anymore?

CPU times: user 301 ms, sys: 50.2 ms, total: 352 ms
Wall time: 4.05 s


,PublicTransactionId,TransactionId,TransactionDate,TransactionTypeId,TransactionTypeDispositionId,CommitteeId,CommitteeUniqueId,CommitteeName,Amount,EntityTypeId,...,TransactionState,TransactionZipCode,EntityDescription,Memo,TransactionType,JurisdictionId,JurisdictionName,ReportId,TransactionDateYear,BenefitedOpposed
0,5006439,9266081,/Date(1628060400000)/,1,1,100677,50411,Morgan Abraham For State Representative - Dist...,25.00,10,...,AZ,85711,Individual,None,Contribution from Candidate or Family,0,Arizona Secretary of State,None,2021,Opposed
1,5006440,9265969,/Date(1629183600000)/,2,1,100677,50411,Morgan Abraham For State Representative - Dist...,2900.00,10,...,AZ,85719,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2021,Opposed
2,5006441,9265976,/Date(1629874800000)/,2,1,100677,50411,Morgan Abraham For State Representative - Dist...,2650.00,10,...,AZ,85711,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2021,Opposed
3,5006442,9265987,/Date(1629961200000)/,1,1,100677,50411,Morgan Abraham For State Representative - Dist...,500.00,10,...,AZ,85260,Individual,None,Contribution from Candidate or Family,0,Arizona Secretary of State,None,2021,Opposed
4,5006443,9266047,/Date(1630220400000)/,1,1,100677,50411,Morgan Abraham For State Representative - Dist...,500.00,10,...,AZ,85260,Individual,None,Contribution from Candidate or Family,0,Arizona Secretary of State,None,2021,Opposed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,4619507,9540100,/Date(1640156400000)/,31,2,100568,49951,Dana Allmond For State Representative - Distri...,67.72,10,...,AZ,85004,Business,None,Operating Expense - Pay Cash/Check,0,Arizona Secretary of State,None,2021,Opposed
323,4619508,9336017,/Date(1640502000000)/,2,1,100568,49951,Dana Allmond For State Representative - Distri...,25.00,10,...,TN,22306,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2021,Opposed
324,4619509,9499318,/Date(1640588400000)/,2,1,100568,49951,Dana Allmond For State Representative - Distri...,50.00,10,...,MD,21113,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2021,Opposed
325,4619510,9336019,/Date(1640761200000)/,2,1,100568,49951,Dana Allmond For State Representative - Distri...,25.00,10,...,WA,98282,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2021,Opposed


In [276]:
all_trans_cands.columns

#okay, that's a lot of columns we don't need
#was it the function of the base stuff to restrict this?
#I suppose not

Index(['PublicTransactionId', 'TransactionId', 'TransactionDate',
       'TransactionTypeId', 'TransactionTypeDispositionId', 'CommitteeId',
       'CommitteeUniqueId', 'CommitteeName', 'Amount', 'EntityTypeId',
       'SubjectCommitteeId', 'SubjectCommitteeName', 'IsForBenefit',
       'BallotMeasureNumber', 'CandidateCycleId', 'CandidateOfficeTypeId',
       'CandidateOfficeId', 'CandidatePartyId', 'CandidateFirstName',
       'CandidateMiddleName', 'CandidateLastName', 'TransactionNameId',
       'TransactionNameGroupId', 'TransactionEntityTypeId',
       'TransactionFirstName', 'TransactionMiddleName', 'TransactionLastName',
       'ReceivedFromOrPaidTo', 'TransactionOccupation', 'TransactionEmployer',
       'SubjectCommitteeNameId', 'TransactionDateYearMonth', 'BallotMeasureId',
       'TransactionGroupNumber', 'TransactionGroupName',
       'TransactionGroupColor', 'CommitteeGroupNumber', 'CommitteeGroupName',
       'CommitteeGroupColor', 'SubjectGroupNumber', 'TransactionCity'

In [272]:


base24 = {
    'draw': '2',
    'columns[0][data]': '',
    'columns[0][name]': '',
    'columns[0][searchable]': 'true',
    'columns[0][orderable]': 'false',
    'columns[0][search][value]': '',
    'columns[0][search][regex]': 'false',
    'columns[1][data]': 'TransactionId',
    'columns[1][name]': '',
    'columns[1][searchable]': 'true',
    'columns[1][orderable]': 'true',
    'columns[1][search][value]': '',
    'columns[1][search][regex]': 'false',
    'columns[2][data]': 'TransactionDate',
    'columns[2][name]': '',
    'columns[2][searchable]': 'true',
    'columns[2][orderable]': 'true',
    'columns[2][search][value]': '',
    'columns[2][search][regex]': 'false',
    'columns[3][data]': 'Amount',
    'columns[3][name]': '',
    'columns[3][searchable]': 'true',
    'columns[3][orderable]': 'true',
    'columns[3][search][value]': '',
    'columns[3][search][regex]': 'false',
    'columns[4][data]': 'TransactionType',
    'columns[4][name]': '',
    'columns[4][searchable]': 'true',
    'columns[4][orderable]': 'true',
    'columns[4][search][value]': '',
    'columns[4][search][regex]': 'false',
    'columns[5][data]': 'ReceivedFromOrPaidTo',
    'columns[5][name]': '',
    'columns[5][searchable]': 'true',
    'columns[5][orderable]': 'true',
    'columns[5][search][value]': '',
    'columns[5][search][regex]': 'false',
    'order[0][column]': '2',
    'order[0][dir]': 'desc',
    'start': '0',
    'length': '10',
    'search[value]': '',
    'search[regex]': 'false',
}


In [268]:
response = requests.post(
    'https://seethemoney.az.gov/Reporting/GetNEWDetailedTableData/',
    params=pars[0],
    #cookies=cookies,
    headers=head,
    data=base1,
)

res = response.json()
pd.DataFrame(data = res["data"])

""


In [329]:
%%time
pa = az_wrapper("PAC/IEFor", 2019, 2019)

CPU times: user 16.5 s, sys: 1.99 s, total: 18.5 s
Wall time: 3min 31s


In [331]:
import re
from datetime import datetime

# Input string with the date format
date_str = "/Date(1563519600000)/"

# Extract the Unix timestamp using regular expressions
timestamp_match = re.match(r"/Date\((\d+)\)/", date_str)

if timestamp_match:
    timestamp = int(timestamp_match.group(1))
    # Convert the Unix timestamp to seconds (divide by 1000) and create a datetime object
    date = datetime.utcfromtimestamp(timestamp / 1000)
    print(date)
else:
    print("Invalid date format")


2019-07-19 07:00:00


In [334]:
#dates as unix timestamps


# Define a function to convert the date string to a datetime object
def convert_date(date_str):
    timestamp_match = re.match(r"/Date\((\d+)\)/", date_str)
    if timestamp_match:
        timestamp = int(timestamp_match.group(1))
        return datetime.utcfromtimestamp(timestamp / 1000)
    else:
        return None  # Return None for invalid date formats

# # Apply the function to the entire column
# df['DateColumn'] = df['DateColumn'].apply(convert_date)

# print(df)

In [385]:
convert_date("/Date(1561964400000)/")

datetime.datetime(2019, 7, 1, 7, 0)

In [ ]:
#let's try to figure out cleaning steps
pa["TransactionDate"] = pa["TransactionDate"].apply(convert_date) #these times are all 7 am bc 

In [378]:
pa["ReceivedFromOrPaidTo"][500]#.iloc[:, 20: 30]

#format: number | name ||| shorter number | very short number | number ||| name

#it's not the page number and sub-number, they don't match up, 
#there is no sub-page 44
# the | aren't empty, they're separators, sometimes the 
#thing they're supposed to be separating isn't there
#the second number is probably the transaction group number
#not sure what that is either

'1655865|Grubhub|||44|8|1655865|||Grubhub'

In [384]:
pa.iloc[:, 30 : 40]

,SubjectCommitteeNameId,TransactionDateYearMonth,BallotMeasureId,TransactionGroupNumber,TransactionGroupName,TransactionGroupColor,CommitteeGroupNumber,CommitteeGroupName,CommitteeGroupColor,SubjectGroupNumber
0,None,/Date(1561964400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
1,None,/Date(1561964400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
2,None,/Date(1561964400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
3,None,/Date(1561964400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
4,None,/Date(1561964400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
...,...,...,...,...,...,...,...,...,...,...
600,None,/Date(1559372400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
601,None,/Date(1559372400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
602,None,/Date(1561964400000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None
603,1667207,/Date(1569913200000)/,None,8,Vendors,ac8c55,2,PACs,cf9c51,None


In [ ]:
#will have to find or make a dictionary of transaction types etc

# 1: Contribution from Candidate or Family
# 2: Contribution from Individuals
# 3: Contribution from Committee
# 5: In-State Contributions $100 or Less
# 31: Operating Expense - Pay Cash/Check 
# 36: Pay Another Committee for Shared Expense
#64: Individual In-Kind Contribution
# 99: Expend Individual In-Kind Contribution

# 215: Ind. Expend. (Non-Recall) - cash

In [395]:
ls = az["ReceivedFromOrPaidTo"].str.split("|")#[az["TransactionTypeId"]==7]

In [407]:
ls[0]

['1368340', 'Ross', 'Mairi', '', '1', '7', '1368340', '', '', 'Ross, Mairi']

In [391]:
az.iloc[:, 20 : 30]#.columns#[100:120]

,CandidateLastName,TransactionNameId,TransactionNameGroupId,TransactionEntityTypeId,TransactionFirstName,TransactionMiddleName,TransactionLastName,ReceivedFromOrPaidTo,TransactionOccupation,TransactionEmployer
0,Bagley,1368340,1368340,1,Mairi,None,"Ross, Mairi","1368340|Ross|Mairi||1|7|1368340|||Ross, Mairi",Product Development Writer,Self- Ind Contractor
1,Bagley,1259548,1259548,1,Linda,None,"Buchanan, Linda",1259548|Buchanan|Linda||1|7|1259548|||Buchanan...,Yavapai College,None
2,Bagley,1368342,1368342,3,None,None,Office Depot Office Max,1368342|Office Depot Office Max|||3|8|1368342|...,None,None
3,Bagley,1368345,1368345,3,None,None,Clarkdale Post Office,1368345|Clarkdale Post Office|||3|8|1368345|||...,None,None
4,Bagley,678003,678003,3,None,None,Deluxe Check,678003|Deluxe Check|||3|8|678003|||Deluxe Check,None,None
...,...,...,...,...,...,...,...,...,...,...
62306,Yee,842603,842603,3,None,None,Paypal,842603|Paypal|||3|8|842603|||Paypal,None,None
62307,Yee,1440239,1050035,42,None,None,Arizona Chamber Political Action Committee,1440239|Arizona Chamber Political Action Commi...,None,None
62308,Yee,1442262,696921,42,None,None,Salt River Project Political Involvement Commi...,1442262|Salt River Project Political Involveme...,None,None
62309,Yee,1442259,695509,42,None,None,Pinnacle West Capital Corporation Political Ac...,1442259|Pinnacle West Capital Corporation Poli...,None,None


In [339]:
az = az_wrapper("Candidate/All Transactions", 2016, 2016)

#what is the diff btwn public transaction id and transactions id?

#useful columns: transaction id
#transaction date, transaction type, amount, committee name, amount, entity description,

#Receivedfromorpaidto is a useful column, but unclear how it's formatted
#one number is transaction name id, another in transaction group id, but the second
#is unclear
#still, looks like it's redundant with other stuff, except the strings, which 
#can be pulled out just from the last element
#considering the cleanliness of the dataset, I don't think we need to worry about the nulls
#we just expect a bunch of them for this, it's part of the landscape

#why are so many listed as Opposing in the final column?

,PublicTransactionId,TransactionId,TransactionDate,TransactionTypeId,TransactionTypeDispositionId,CommitteeId,CommitteeUniqueId,CommitteeName,Amount,EntityTypeId,...,TransactionState,TransactionZipCode,EntityDescription,Memo,TransactionType,JurisdictionId,JurisdictionName,ReportId,TransactionDateYear,BenefitedOpposed
0,7723778,6228722,/Date(1451631600000)/,2,1,201600383,37603,Nikki Bagley Ld6 Campaign,1000.00,10,...,AZ,86331,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2016,Opposed
1,7723779,6546318,/Date(1451977200000)/,2,1,201600383,37603,Nikki Bagley Ld6 Campaign,1000.00,10,...,AZ,86322,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2016,Opposed
2,7723780,6228729,/Date(1452150000000)/,31,2,201600383,37603,Nikki Bagley Ld6 Campaign,27.23,10,...,AZ,86326,Business,copies,Operating Expense - Pay Cash/Check,0,Arizona Secretary of State,None,2016,Opposed
3,7723781,6228734,/Date(1452236400000)/,31,2,201600383,37603,Nikki Bagley Ld6 Campaign,1.42,10,...,AZ,86324,Business,petitions mailed,Operating Expense - Pay Cash/Check,0,Arizona Secretary of State,None,2016,Opposed
4,7723782,6546326,/Date(1452582000000)/,31,2,201600383,37603,Nikki Bagley Ld6 Campaign,20.75,10,...,MN,55164,Business,printing checks,Operating Expense - Pay Cash/Check,0,Arizona Secretary of State,None,2016,Opposed
5,7723783,6228723,/Date(1452668400000)/,2,1,201600383,37603,Nikki Bagley Ld6 Campaign,2000.00,10,...,AZ,86322,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2016,Opposed
6,7723784,6547336,/Date(1452841200000)/,31,2,201600383,37603,Nikki Bagley Ld6 Campaign,18.57,10,...,AZ,86326,Business,clipboards,Operating Expense - Pay Cash/Check,0,Arizona Secretary of State,None,2016,Opposed
7,7723785,6290078,/Date(1452927600000)/,31,2,201600383,37603,Nikki Bagley Ld6 Campaign,142.14,10,...,AZ,86326,Business,telephone (2-line),Operating Expense - Pay Cash/Check,0,Arizona Secretary of State,None,2016,Opposed
8,7723786,6260588,/Date(1453014000000)/,31,2,201600383,37603,Nikki Bagley Ld6 Campaign,99.89,10,...,FL,33161,Business,committee name badges,Operating Expense - Pay Cash/Check,0,Arizona Secretary of State,None,2016,Opposed
9,7723787,6546319,/Date(1453100400000)/,2,1,201600383,37603,Nikki Bagley Ld6 Campaign,500.00,10,...,Az,86018,Individual,None,Contribution from Individuals,0,Arizona Secretary of State,None,2016,Opposed


In [354]:
#az.groupby(by="TransactionTypeId").count()
az.groupby(by="EntityDescription").count()

,PublicTransactionId,TransactionId,TransactionDate,TransactionTypeId,TransactionTypeDispositionId,CommitteeId,CommitteeUniqueId,CommitteeName,Amount,EntityTypeId,...,TransactionCity,TransactionState,TransactionZipCode,Memo,TransactionType,JurisdictionId,JurisdictionName,ReportId,TransactionDateYear,BenefitedOpposed
EntityDescription,,,,,,,,,,,,,,,,,,,,,
Business,14934,14934,14934,14934,14934,14934,14934,14934,14934,14934,...,14934,14934,14934,5571,14934,14934,14934,0,14934,14934
Business Vendor,4,4,4,4,4,4,4,4,4,4,...,2,4,2,4,4,4,4,0,4,4
Candidate (Exploratory),3,3,3,3,3,3,3,3,3,3,...,3,3,3,0,3,3,3,0,3,3
Candidate (not participating in Clean Elections),136,136,136,136,136,136,136,136,136,136,...,136,136,136,57,136,136,136,0,136,136
Candidate (participating in Clean Elections),74,74,74,74,74,74,74,74,74,74,...,74,74,74,67,74,74,74,0,74,74
Independent Exp. (Standing) (Multicandidate PAC),44,44,44,44,44,44,44,44,44,44,...,44,44,44,12,44,44,44,0,44,44
Independent Expenditures,64,64,64,64,64,64,64,64,64,64,...,64,64,64,10,64,64,64,0,64,64
Independent Expenditures (Corp/LLC/Labor Org),17,17,17,17,17,17,17,17,17,17,...,0,0,0,1,17,17,17,0,17,17
Independent Expenditures (Standing),3,3,3,3,3,3,3,3,3,3,...,3,3,3,1,3,3,3,0,3,3


In [269]:
pars

[{'Page': '24',
  'startYear': '2021',
  'endYear': '2021',
  'JurisdictionId': '0|Page',
  'TablePage': '1',
  'TableLength': '100000',
  'Name': '1~100994',
  'entityId': '100994',
  'ChartName': '24',
  'IsLessActive': 'false',
  'ShowOfficeHolder': 'false'},
 {'Page': '24',
  'startYear': '2021',
  'endYear': '2021',
  'JurisdictionId': '0|Page',
  'TablePage': '1',
  'TableLength': '100000',
  'Name': '1~100894',
  'entityId': '100894',
  'ChartName': '24',
  'IsLessActive': 'false',
  'ShowOfficeHolder': 'false'},
 {'Page': '24',
  'startYear': '2021',
  'endYear': '2021',
  'JurisdictionId': '0|Page',
  'TablePage': '1',
  'TableLength': '100000',
  'Name': '1~100894',
  'entityId': '100894',
  'ChartName': '24',
  'IsLessActive': 'false',
  'ShowOfficeHolder': 'false'},
 {'Page': '24',
  'startYear': '2021',
  'endYear': '2021',
  'JurisdictionId': '0|Page',
  'TablePage': '1',
  'TableLength': '100000',
  'Name': '1~100883',
  'entityId': '100883',
  'ChartName': '24',
  'IsLe

## Base

In [166]:
head = {
    'authority': 'seethemoney.az.gov',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.7',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': 'SeeTheMoneyUserHistory=UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=24|startYear=2002|endYear=2023|IsLessActive=false|ShowOfficeHolder=false|View=Detail|Name=1~201600383|entityId=201600383|TablePage=1|TableLength=10; __cf_bm=KM9k.9h98I0bz.4.pDYgx1g.Qoye0zUW.nD8f1erLyQ-1698336502-0-AYeHoY9FQCJetaiH2kyxcHYRD0Cs7+ZDGsyznhAc52/uxZNDjMQ3b3vK196Z5WB4agNnZdzz5IvrsNIE6JxEzSk=',
    'origin': 'https://seethemoney.az.gov',
    'referer': 'https://seethemoney.az.gov/Reporting/Explore',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

In [263]:
base1 = { 
    'draw': '2',
    'order[0][column]': '0',
    'order[0][dir]': 'asc',
    'start': '0',
    'length': '100000',
    'search[value]': '',
    'search[regex]': 'false',
}

base2 = {   
    'draw': '2',
    'order[0][column]': '2',
    'order[0][dir]': 'asc',
    'start': '0',
    'length': '100000',
    'search[value]': '',
    'search[regex]': 'false',
}


#wait, if these all turn out to be the same....


In [282]:
import requests

cookies = {
    'SeeTheMoneyUserHistory': 'UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=1|startYear=2023|endYear=2025|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10',
    '__cf_bm': 'wK034ix7QzZ2yMZlLAg2xfUtoCWpvarGaRguGymVNuE-1698343321-0-AZw++KGNFiVyY+ABzBQ5eVsFg9uWQHd01msDi2prfjAT3HBNA/8oj78TlyKPAD9aTwNU46qrAi69PzgH6vx7dzM=',
}

headers = {
    'authority': 'seethemoney.az.gov',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.7',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': 'SeeTheMoneyUserHistory=UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=1|startYear=2023|endYear=2025|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10; __cf_bm=wK034ix7QzZ2yMZlLAg2xfUtoCWpvarGaRguGymVNuE-1698343321-0-AZw++KGNFiVyY+ABzBQ5eVsFg9uWQHd01msDi2prfjAT3HBNA/8oj78TlyKPAD9aTwNU46qrAi69PzgH6vx7dzM=',
    'origin': 'https://seethemoney.az.gov',
    'referer': 'https://seethemoney.az.gov/Reporting/Explore',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

params = [
    ('Page', '1'),
    ('startYear', '2023'),
    ('endYear', '2025'),
    ('JurisdictionId', '0|Page'),
    ('TablePage', '1'),
    ('TableLength', '10'),
    ('IsLessActive', 'false'),
    ('ShowOfficeHolder', 'false'),
    ('ChartName', '1'),
    ('IsLessActive', 'false'),
    ('ShowOfficeHolder', 'false'),
]

data = {
#     'draw': '2',
    
#     'columns[0][data]': 'EntityLastName',
#     'columns[0][name]': '',
# #     'columns[0][searchable]': 'true',
# #     'columns[0][orderable]': 'true',
# #     'columns[0][search][value]': '',
# #     'columns[0][search][regex]': 'false',
    
#     'columns[1][data]': 'CommitteeName',
#     'columns[1][name]': '',
# #     'columns[1][searchable]': 'true',
# #     'columns[1][orderable]': 'true',
# #     'columns[1][search][value]': '',
# #     'columns[1][search][regex]': 'false',
    
#     'columns[2][data]': 'OfficeName',
#     'columns[2][name]': '',
# #     'columns[2][searchable]': 'true',
# #     'columns[2][orderable]': 'true',
# #     'columns[2][search][value]': '',
# #     'columns[2][search][regex]': 'false',
    
#     'columns[3][data]': 'PartyName',
#     'columns[3][name]': '',
# #     'columns[3][searchable]': 'true',
# #     'columns[3][orderable]': 'true',
# #     'columns[3][search][value]': '',
# #     'columns[3][search][regex]': 'false',
    
#     'columns[4][data]': 'Income',
#     'columns[4][name]': '',
# #     'columns[4][searchable]': 'true',
# #     'columns[4][orderable]': 'true',
# #     'columns[4][search][value]': '',
# #     'columns[4][search][regex]': 'false',
    
#     'columns[5][data]': 'Expense',
#     'columns[5][name]': '',
# #     'columns[5][searchable]': 'true',
# #     'columns[5][orderable]': 'true',
# #     'columns[5][search][value]': '',
# #     'columns[5][search][regex]': 'false',
    
#     'columns[6][data]': 'CashBalance',
#     'columns[6][name]': '',
# #     'columns[6][searchable]': 'true',
# #     'columns[6][orderable]': 'true',
# #     'columns[6][search][value]': '',
# #     'columns[6][search][regex]': 'false',
    
#     'columns[7][data]': 'IESupport',
#     'columns[7][name]': '',
# #     'columns[7][searchable]': 'true',
# #     'columns[7][orderable]': 'true',
# #     'columns[7][search][value]': '',
# #     'columns[7][search][regex]': 'false',
    
#     'columns[8][data]': 'IEOpposition',
#     'columns[8][name]': '',
# #     'columns[8][searchable]': 'true',
# #     'columns[8][orderable]': 'true',
# #     'columns[8][search][value]': '',
# #     'columns[8][search][regex]': 'false',
    
    'order[0][column]': '0',
    'order[0][dir]': 'asc',
    'start': '0',
    'length': '10',
    'search[value]': '',
    'search[regex]': 'false',
}

response = requests.post(
    'https://seethemoney.az.gov/Reporting/GetNEWTableData/',
    params=params,
    cookies=cookies,
    headers=headers,
    data=data,
)

In [283]:
results = response.json()
test_df2 = pd.DataFrame(data=results["data"])
test_df2.head()

,EntityID,EntityFirstName,EntityLastName,Income,Expense,IESupport,IEOpposition,OfficeID,PartyID,OfficeTypeID,OfficeName,PartyName,CashBalance,BMEFor,BMEAgainst,EntityTypeName,CommitteeName
0,100894,None,"Abdussamad, Shams <br>(100894)",0.0,0.0,0.0,0.0,3211,10.0,None,State Representative - District 11,Democratic,0.00,0.0,0.0,Candidate (participating in Clean Elections),Shams For AZ<br>(Clean Elections)
1,202000060,None,"Alston, Lela <br>(202000060)",0.0,0.0,0.0,0.0,3105,10.0,None,State Senator - District No. 5,Democratic,65797.06,0.0,0.0,Candidate (not participating in Clean Elections),Lela Alston AZ Senate 2024
2,101421,None,"Altherr, Michelle <br>(101421)",0.0,0.0,0.0,0.0,3223,30.0,None,State Representative - District 23,Republican,0.00,0.0,0.0,Candidate (participating in Clean Elections),Michelle Altherr for State Representative - Di...
3,101475,None,"Altherr, Michelle <br>(101475)",0.0,0.0,0.0,0.0,3123,NaN,None,State Senator - District No. 23,,0.00,0.0,0.0,Candidate (not participating in Clean Elections),Elect Michelle Altherr
4,101430,None,"Angius, Hildy L<br>(101430)",0.0,0.0,0.0,0.0,3130,NaN,None,State Senator - District No. 30,,0.00,0.0,0.0,Candidate (not participating in Clean Elections),Hildy Angius for State Senator - District No. 30


In [286]:
type(response)

requests.models.Response

In [61]:
import requests

# cookies = {
#     '__cf_bm': 'P70WvhIsRik5WvJ8GgNBVypADvKYB46zFMfvDiZnuxo-1698337703-0-AQyumSSNxFlDoJKCJN0/fe3xJUGv3bIGf0HR9oTSKhvSOjcrQdrR1pfcwM9ETEg1SYXNS097c+ElJfuotuzZVlk=',
#     'SeeTheMoneyUserHistory': 'UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=1|startYear=2023|endYear=2025|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10',
# }

headers = {
    'authority': 'seethemoney.az.gov',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.7',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': '__cf_bm=P70WvhIsRik5WvJ8GgNBVypADvKYB46zFMfvDiZnuxo-1698337703-0-AQyumSSNxFlDoJKCJN0/fe3xJUGv3bIGf0HR9oTSKhvSOjcrQdrR1pfcwM9ETEg1SYXNS097c+ElJfuotuzZVlk=; SeeTheMoneyUserHistory=UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=1|startYear=2023|endYear=2025|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10',
    'origin': 'https://seethemoney.az.gov',
    'referer': 'https://seethemoney.az.gov/Reporting/Explore',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

# params = [
#     ('Page', '1'),
#     ('startYear', '2022'),
#     ('endYear', '2022'),
#     ('JurisdictionId', '0|Page'),
#     ('TablePage', '1'),
#     ('TableLength', '10'),
#     ('IsLessActive', 'false'),
#     ('ShowOfficeHolder', 'false'),
#     ('ChartName', '1'),
#     ('IsLessActive', 'false'),
#     ('ShowOfficeHolder', 'false'),
# ]

params = {
    'Page': '1',
    'startYear': '2022',
    'endYear': '2022',
    'JurisdictionId': '0|Page',
    'TablePage': '1',
    'TableLength': '10',
    
    'Name': '1~201600383',
    'entityId': '201600383',
    
    'ChartName': '1',

    'IsLessActive': 'false',
    'ShowOfficeHolder': 'false',
}

data = {
    'draw': '2',
    'columns[0][data]': 'EntityLastName',
    'columns[0][name]': '',
    'columns[0][searchable]': 'true',
    'columns[0][orderable]': 'true',
    'columns[0][search][value]': '',
    'columns[0][search][regex]': 'false',
    'columns[1][data]': 'CommitteeName',
    'columns[1][name]': '',
    'columns[1][searchable]': 'true',
    'columns[1][orderable]': 'true',
    'columns[1][search][value]': '',
    'columns[1][search][regex]': 'false',
    'columns[2][data]': 'OfficeName',
    'columns[2][name]': '',
    'columns[2][searchable]': 'true',
    'columns[2][orderable]': 'true',
    'columns[2][search][value]': '',
    'columns[2][search][regex]': 'false',
    'columns[3][data]': 'PartyName',
    'columns[3][name]': '',
    'columns[3][searchable]': 'true',
    'columns[3][orderable]': 'true',
    'columns[3][search][value]': '',
    'columns[3][search][regex]': 'false',
    'columns[4][data]': 'Income',
    'columns[4][name]': '',
    'columns[4][searchable]': 'true',
    'columns[4][orderable]': 'true',
    'columns[4][search][value]': '',
    'columns[4][search][regex]': 'false',
    'columns[5][data]': 'Expense',
    'columns[5][name]': '',
    'columns[5][searchable]': 'true',
    'columns[5][orderable]': 'true',
    'columns[5][search][value]': '',
    'columns[5][search][regex]': 'false',
    'columns[6][data]': 'CashBalance',
    'columns[6][name]': '',
    'columns[6][searchable]': 'true',
    'columns[6][orderable]': 'true',
    'columns[6][search][value]': '',
    'columns[6][search][regex]': 'false',
    'columns[7][data]': 'IESupport',
    'columns[7][name]': '',
    'columns[7][searchable]': 'true',
    'columns[7][orderable]': 'true',
    'columns[7][search][value]': '',
    'columns[7][search][regex]': 'false',
    'columns[8][data]': 'IEOpposition',
    'columns[8][name]': '',
    'columns[8][searchable]': 'true',
    'columns[8][orderable]': 'true',
    'columns[8][search][value]': '',
    'columns[8][search][regex]': 'false',
    'order[0][column]': '0',
    'order[0][dir]': 'asc',
    'start': '0',
    'length': '10',
    'search[value]': '',
    'search[regex]': 'false',
}

response = requests.post(
    'https://seethemoney.az.gov/Reporting/GetNEWTableData/',
    params=params,
    cookies=cookies,
    headers=headers,
    data=data,
)

In [60]:
results = response.json()
test_df2 = pd.DataFrame(data=results["data"])
test_df2

,EntityID,EntityFirstName,EntityLastName,Income,Expense,IESupport,IEOpposition,OfficeID,PartyID,OfficeTypeID,OfficeName,PartyName,CashBalance,BMEFor,BMEAgainst,EntityTypeName,CommitteeName
0,100994,None,"Abdo, Jeremy <br>(100994)",0.00,0.00,0.00,0.00,3203,30.0,None,State Representative - District 3,Republican,0.00,0.0,0.0,Candidate (not participating in Clean Elections),Abdo for Arizona State House LD3
1,100894,None,"Abdussamad, Shams <br>(100894)",26463.01,28428.14,0.00,0.00,3211,10.0,None,State Representative - District 11,Democratic,0.00,0.0,0.0,Candidate (participating in Clean Elections),Shams For AZ<br>(Clean Elections)
2,100894,None,"Abdussamad, Shams <br>(100894)",3800.00,1834.87,2825.92,0.00,3227,10.0,None,State Representative - District 27,Democratic,0.00,0.0,0.0,Candidate (participating in Clean Elections),Shams For AZ<br>(Clean Elections)
3,100883,None,"Abeytia, Anna Lynn<br>(100883)",29230.76,23758.62,0.00,0.00,3224,10.0,None,State Representative - District 24,Democratic,8975.22,0.0,0.0,Candidate (participating in Clean Elections),Abeytia for AZ<br>(Clean Elections)
4,100883,None,"Abeytia, Anna Lynn<br>(100883)",508.97,245.29,7739.99,0.00,3229,10.0,None,State Representative - District 29,Democratic,8975.22,0.0,0.0,Candidate (participating in Clean Elections),Abeytia for AZ<br>(Clean Elections)
5,100677,None,"Abraham, Morgan <br>(100677)",192960.00,294741.32,0.00,0.00,3118,10.0,None,State Senator - District No. 18,Democratic,4363.10,0.0,0.0,Candidate (not participating in Clean Elections),Morgan for Arizona
6,100677,None,"Abraham, Morgan <br>(100677)",6786.00,824.96,116173.71,0.00,3210,10.0,None,State Representative - District 10,Democratic,4363.10,0.0,0.0,Candidate (not participating in Clean Elections),Morgan for Arizona
7,100986,None,"Aguilar, Cesar <br>(100986)",33323.01,33315.86,5463.84,0.00,3226,10.0,None,State Representative - District 26,Democratic,7.15,0.0,0.0,Candidate (participating in Clean Elections),Cesar Aguilar for State Representative - Distr...
8,101037,None,"Alger, David M<br>(101037)",0.00,0.00,0.00,0.00,3208,NaN,None,State Representative - District 8,,0.00,0.0,0.0,Candidate (not participating in Clean Elections),David Alger for State Representative - District 8
9,100568,None,"Allmond, Dana <br>(100568)",71142.93,79943.05,77802.58,3887.52,3211,10.0,None,State Representative - District 11,Democratic,47827.62,0.0,0.0,Candidate (not participating in Clean Elections),Dana Allmond for State Representative - Distri...


In [62]:
results = response.json()
test_df = pd.DataFrame(data=results["data"])
test_df

,EntityID,EntityFirstName,EntityLastName,Income,Expense,IESupport,IEOpposition,OfficeID,PartyID,OfficeTypeID,OfficeName,PartyName,CashBalance,BMEFor,BMEAgainst,EntityTypeName,CommitteeName
0,100994,None,"Abdo, Jeremy <br>(100994)",0.00,0.00,0.00,0.00,3203,30.0,None,State Representative - District 3,Republican,0.00,0.0,0.0,Candidate (not participating in Clean Elections),Abdo for Arizona State House LD3
1,100894,None,"Abdussamad, Shams <br>(100894)",26463.01,28428.14,0.00,0.00,3211,10.0,None,State Representative - District 11,Democratic,0.00,0.0,0.0,Candidate (participating in Clean Elections),Shams For AZ<br>(Clean Elections)
2,100894,None,"Abdussamad, Shams <br>(100894)",3800.00,1834.87,2825.92,0.00,3227,10.0,None,State Representative - District 27,Democratic,0.00,0.0,0.0,Candidate (participating in Clean Elections),Shams For AZ<br>(Clean Elections)
3,100883,None,"Abeytia, Anna Lynn<br>(100883)",29230.76,23758.62,0.00,0.00,3224,10.0,None,State Representative - District 24,Democratic,8975.22,0.0,0.0,Candidate (participating in Clean Elections),Abeytia for AZ<br>(Clean Elections)
4,100883,None,"Abeytia, Anna Lynn<br>(100883)",508.97,245.29,7739.99,0.00,3229,10.0,None,State Representative - District 29,Democratic,8975.22,0.0,0.0,Candidate (participating in Clean Elections),Abeytia for AZ<br>(Clean Elections)
5,100677,None,"Abraham, Morgan <br>(100677)",192960.00,294741.32,0.00,0.00,3118,10.0,None,State Senator - District No. 18,Democratic,4363.10,0.0,0.0,Candidate (not participating in Clean Elections),Morgan for Arizona
6,100677,None,"Abraham, Morgan <br>(100677)",6786.00,824.96,116173.71,0.00,3210,10.0,None,State Representative - District 10,Democratic,4363.10,0.0,0.0,Candidate (not participating in Clean Elections),Morgan for Arizona
7,100986,None,"Aguilar, Cesar <br>(100986)",33323.01,33315.86,5463.84,0.00,3226,10.0,None,State Representative - District 26,Democratic,7.15,0.0,0.0,Candidate (participating in Clean Elections),Cesar Aguilar for State Representative - Distr...
8,101037,None,"Alger, David M<br>(101037)",0.00,0.00,0.00,0.00,3208,NaN,None,State Representative - District 8,,0.00,0.0,0.0,Candidate (not participating in Clean Elections),David Alger for State Representative - District 8
9,100568,None,"Allmond, Dana <br>(100568)",71142.93,79943.05,77802.58,3887.52,3211,10.0,None,State Representative - District 11,Democratic,47827.62,0.0,0.0,Candidate (not participating in Clean Elections),Dana Allmond for State Representative - Distri...


In [79]:
%%time

# cookies = {
#     '__cf_bm': '_mM_emhAfMVfIggnwHkSz7UVoEl8uu6VElhCijwr5Lc-1698340318-0-AbjC03NmOHga8Oqa2Bw1QLr0EeaGEqj2ot341xWm1fBFfkGUKQywqbaitA44wUoFdVG+q3sAVlhEDTEp8mn3dps=',
#     'SeeTheMoneyUserHistory': 'UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=7|startYear=2022|endYear=2022|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10',
# }

headers = {
    'authority': 'seethemoney.az.gov',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.7',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': '__cf_bm=_mM_emhAfMVfIggnwHkSz7UVoEl8uu6VElhCijwr5Lc-1698340318-0-AbjC03NmOHga8Oqa2Bw1QLr0EeaGEqj2ot341xWm1fBFfkGUKQywqbaitA44wUoFdVG+q3sAVlhEDTEp8mn3dps=; SeeTheMoneyUserHistory=UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=7|startYear=2022|endYear=2022|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10',
    'origin': 'https://seethemoney.az.gov',
    'referer': 'https://seethemoney.az.gov/Reporting/Explore',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

params = [
    ('Page', '7'),
    ('startYear', '2002'),
    ('endYear', '2005'),
    ('JurisdictionId', '0|Page'),
    ('TablePage', '1'),
    ('TableLength', '10'),
    ('IsLessActive', 'false'),
    ('ShowOfficeHolder', 'false'),
    ('ChartName', '7'),
    ('IsLessActive', 'false'),
    ('ShowOfficeHolder', 'false'),
]

data = {
    'draw': '2',
    'columns[0][data]': 'EntityLastName',
    'columns[0][name]': '',
    'columns[0][searchable]': 'true',
    'columns[0][orderable]': 'true',
    'columns[0][search][value]': '',
    'columns[0][search][regex]': 'false',
    'columns[1][data]': 'Income',
    'columns[1][name]': '',
    'columns[1][searchable]': 'true',
    'columns[1][orderable]': 'true',
    'columns[1][search][value]': '',
    'columns[1][search][regex]': 'false',
    'order[0][column]': '1',
    'order[0][dir]': 'desc',
    'start': '0',
    'length': '10',
    'search[value]': '',
    'search[regex]': 'false',
}

response = requests.post(
    'https://seethemoney.az.gov/Reporting/GetNEWTableData/',
    params=params,
    cookies=cookies,
    headers=headers,
    data=data,
)

CPU times: user 23.1 ms, sys: 4.02 ms, total: 27.1 ms
Wall time: 954 ms


In [121]:
%%time

# cookies = {
#     '__cf_bm': '_mM_emhAfMVfIggnwHkSz7UVoEl8uu6VElhCijwr5Lc-1698340318-0-AbjC03NmOHga8Oqa2Bw1QLr0EeaGEqj2ot341xWm1fBFfkGUKQywqbaitA44wUoFdVG+q3sAVlhEDTEp8mn3dps=',
#     'SeeTheMoneyUserHistory': 'UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=7|startYear=2022|endYear=2022|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10',
# }

headers = {
    'authority': 'seethemoney.az.gov',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.7',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': '__cf_bm=_mM_emhAfMVfIggnwHkSz7UVoEl8uu6VElhCijwr5Lc-1698340318-0-AbjC03NmOHga8Oqa2Bw1QLr0EeaGEqj2ot341xWm1fBFfkGUKQywqbaitA44wUoFdVG+q3sAVlhEDTEp8mn3dps=; SeeTheMoneyUserHistory=UserID=5dd5ae80-80ed-45f4-99b3-3d19a324102a&URLHash=JurisdictionId=0|Page|Page=7|startYear=2022|endYear=2022|IsLessActive=false|ShowOfficeHolder=false|View=Detail|TablePage=1|TableLength=10',
    'origin': 'https://seethemoney.az.gov',
    'referer': 'https://seethemoney.az.gov/Reporting/Explore',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

params = {
    'Page': '7',
    'startYear': '2022',
    'endYear': '2022',
    'JurisdictionId': '0|Page',
    'TablePage': '1',
    'ChartName': '7',
    'IsLessActive': 'false',
    'ShowOfficeHolder': 'false',
    'TableLength': '5000',
    
#     'Name': '1~201600383',
#     'entityId': '201600383',
    

}

# params = [
#     ('Page', '7'),
#     ('startYear', '2022'),
#     ('endYear', '2022'),
#     ('JurisdictionId', '0|Page'),
#     ('TablePage', '1'),
#     ('TableLength', '500000'),
#     ('IsLessActive', 'false'),
#     ('ShowOfficeHolder', 'false'),
#     ('ChartName', '7'),
#     ('IsLessActive', 'false'),
#     ('ShowOfficeHolder', 'false'),
# ]

data = {
    'draw': '2',
    'columns[0][data]': 'EntityLastName',
    'columns[0][name]': '',
    'columns[0][searchable]': 'true',
    'columns[0][orderable]': 'true',
    'columns[0][search][value]': '',
    'columns[0][search][regex]': 'false',
    'columns[1][data]': 'Income',
    'columns[1][name]': '',
    'columns[1][searchable]': 'true',
    'columns[1][orderable]': 'true',
    'columns[1][search][value]': '',
    'columns[1][search][regex]': 'false',
    'order[0][column]': '1',
    'order[0][dir]': 'desc',
    'start': '0',
    'length': '5000',
    'search[value]': '',
    'search[regex]': 'false',
}

response = requests.post(
    'https://seethemoney.az.gov/Reporting/GetNEWTableData/',
    params=params,
    cookies=cookies,
    headers=headers,
    data=data,
)

CPU times: user 42.3 ms, sys: 47.9 ms, total: 90.2 ms
Wall time: 4.83 s


In [313]:
results = response.json()
test_df2 = pd.DataFrame(data=results["data"])
test_df2["EntityID"].dtype

dtype('int64')